# Handwritten Digit Generation using GANs

![alt text](https://files.oaiusercontent.com/file-ykBbu8YpLZpoEC63YHeDrJyS?se=2024-09-01T22%3A12%3A08Z&sp=r&sv=2024-08-04&sr=b&rscc=max-age%3D604800%2C%20immutable%2C%20private&rscd=attachment%3B%20filename%3Da1a891f3-7d20-4289-a070-e1d892595996.webp&sig=aJ0siNe03HOcm3CTOFdTOJgqVeoya/POKb%2BsqSi/0CE%3D)

![alt text](https://files.oaiusercontent.com/file-a3hsj79xB0aHT7QoYosXTmJl?se=2024-09-01T22%3A14%3A58Z&sp=r&sv=2024-08-04&sr=b&rscc=max-age%3D604800%2C%20immutable%2C%20private&rscd=attachment%3B%20filename%3D94f8d222-5346-4513-9543-02a0bfa8654f.webp&sig=wH1R24RZAqQv10ZZiEtAWnKllPTOib/zfJRgLphRxiQ%3D)

During my master's at AIMS, I presented on Generative Adversarial Networks (GANs) as part of the Computer Vision class, where I explored the core concepts and applications of GANs in generating realistic images. [https://docs.google.com/presentation/d/1-Do_F4kC8R0UAXQqsL7xn3I84g2oeyqY16m-poOc39c/edit?usp=sharing] . This project will be an extension of that presentation, where I will implement a GAN to generate synthetic handwritten digits.

Through this project, I aim to :

- Develop a Generative Adversarial Network (GAN) to generate synthetic handwritten digits resembling those in the MNIST dataset.
- Explore the foundational principles of generative models.
- Focus on how GANs learn to create new, realistic data based on training examples.
- Gain hands-on experience with GAN architecture and training processes.

## Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image

import os

In [ ]:
# Create directory to save generated images
os.makedirs("gan_images", exist_ok=True)

## Define the Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img


# Define the Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity


## Initialize Models and Optimizers

### Hyperparameters

In [ ]:
latent_dim = 100
lr = 0.0002
batch_size = 64
n_epochs = 50

### Initialize models

In [ ]:
generator = Generator(latent_dim)
discriminator = Discriminator()

### Optimizers

In [ ]:
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

### Loss function

In [ ]:
adversarial_loss = nn.BCELoss()

### Check for CUDA

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)
adversarial_loss.to(device)

## Load the MNIST Dataset

In [ ]:
dataloader = DataLoader(
    datasets.MNIST(
        "./data/mnist", train=True, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])
    ),
    batch_size=batch_size, shuffle=True
)

## Training loop

In [ ]:
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        
        # Adversarial ground truths
        valid = torch.ones(imgs.size(0), 1, device=device, requires_grad=False)
        fake = torch.zeros(imgs.size(0), 1, device=device, requires_grad=False)
        
        # Configure input
        real_imgs = imgs.to(device)
        
        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.randn(imgs.size(0), latent_dim, device=device)

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        # Backprop and optimize
        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        # Backprop and optimize
        d_loss.backward()
        optimizer_D.step()

        # Print progress
        if i % 100 == 0:
            print(f"[Epoch {epoch}/{n_epochs}] [Batch {i}/{len(dataloader)}] " \
                  f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")
    
    # Save generated images every epoch
    save_image(gen_imgs.data[:25], f"gan_images/{epoch}.png", nrow=5, normalize=True)
